In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/MyDrive/Diplomatic2/Implementation")
!ls

 AGNN_1			     elliptic_txs_features.csv	'GRL_FINAL (4).ipynb'
 AGNN_2			     GAT_1			 GRL.ipynb
 AGNN_3			     GAT_2			 Node2Vec
 data_balanced_1000.obj      GAT_2_1			 NoteBooks
 data_balanced_500.obj	     GAT_3			 SAGE_1
 data.obj		     GAT_FL_1			 SAGE_2
 DGI			     GAT_RELU_1			 SAGE_3
 elliptic_txs_classes.csv    GAT_TEST			 stellar.obj
 elliptic_txs_edgelist.csv   GRL_2.ipynb


In [ ]:
#PYTORCH INSTALL
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric

In [4]:
import pickle   
import torch
from torch_geometric.data import Data

fileObj = open('data.obj', 'rb')
data = pickle.load(fileObj)
fileObj.close()
print(data)

Data(x=[203769, 165], edge_index=[2, 234355], y=[203769], n_id=[203769], train_mask=[203769], val_mask=[203769], test_mask=[203769])


In [5]:
import copy

data2 = copy.deepcopy(data)


In [6]:
train_index = data.train_mask.tolist()
val_index = data.val_mask.tolist()
test_index = data.test_mask.tolist()
print(type(train_index))
print(type(val_index))
print(type(test_index))

<class 'list'>
<class 'list'>
<class 'list'>


In [7]:
#USE GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [8]:
data = data.to(device)

In [9]:
#EDW NA DW ALLES PERIPTWSEIS GIA BATCHES!!!
#Epishs na dw th lista me tous geitones
from torch_geometric.loader import NeighborLoader

train_loader = NeighborLoader(data,num_neighbors=[-1, -1,-1], shuffle=True,batch_size=203769)

counter = 0
for batch in train_loader:

    print(len(batch.n_id))
    print(len(batch.y))
    print(type(batch))
    print(batch.test_mask)

203769
203769
<class 'torch_geometric.data.data.Data'>
tensor([False, False, False,  ..., False,  True, False], device='cuda:0')


In [ ]:
#SCRIPT 16
#METRICS
!pip install torchmetrics
from torchmetrics.classification import AUROC
from torchmetrics.classification import BinaryPrecision
from torchmetrics.classification import BinaryRecall
from torchmetrics.classification import BinaryConfusionMatrix
from torchmetrics.classification import MulticlassRecall
from torchmetrics.classification import MulticlassPrecision
from torchmetrics.classification import MulticlassAUROC
from torchmetrics.classification import BinaryF1Score
from torchmetrics.classification import MulticlassF1Score
from torchmetrics.classification import Accuracy

confmat = BinaryConfusionMatrix()
recall = MulticlassRecall(num_classes=2, average=None)
precision = MulticlassPrecision(num_classes=2, average=None)
aucroc = MulticlassAUROC(task="multiclass", num_classes=2, thresholds=None)
#f1 = BinaryF1Score(average=None)
f12 = MulticlassF1Score(num_classes=2, average=None)
accuracy = Accuracy(task="multiclass", num_classes=2)

def conf_matrix(pred_y, y):
  #confmat = BinaryConfusionMatrix().to(device)

  return confmat(pred_y, y)

def brecall(pred_y, y):
  #recall = BinaryRecall().to(device)
  #recall = MulticlassRecall(num_classes=2, average=None).to(device)

  return recall(pred_y, y)

def bprecision(pred_y, y):
  #precision = MulticlassPrecision(num_classes=2, average=None).to(device)

  return precision(pred_y, y)

def auc_roc(pred_y, y):
  #aucroc = AUROC(task="binary").to(device)
  #print('AUC ROC',pred_y[:10])
  #print('AUC ROC',y.shape)
  #aucroc = MulticlassAUROC(task="multiclass", num_classes=2).to(device)

  return aucroc(pred_y, y)

def f1score(pred_y, y):
  #metric = BinaryF1Score(average=None).to(device)

  return f12(pred_y, y)


def accuracy_score(pred_y, y):
    
    return accuracy(pred_y,y)

In [11]:
#TEST WITH FINAL LAYER
from torch_geometric.nn import GATConv
import torch.nn.functional as F

layers_GATv2 = [1,2]
embe_GATv2 = [32,64]
loader_GATv2 = [0,1]
heads_GATv2 = [4,8]
weigths_GATv2 = [True,False]
GAT_HPsv2 = []
GAT_HPsv2.append(layers_GATv2)
GAT_HPsv2.append(embe_GATv2)
GAT_HPsv2.append(loader_GATv2)
GAT_HPsv2.append(heads_GATv2)
GAT_HPsv2.append(weigths_GATv2)

class GAT2(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads):
        super().__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads, dropout=0.6)
        # On the Pubmed dataset, use `heads` output heads in `conv2`.
        self.conv2 = GATConv(hidden_channels * heads, out_channels, heads=1,concat=False, dropout=0.6)
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.005, weight_decay=5e-4)  


    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x, F.softmax(x, dim=1) 

In [12]:
def make_train_step(model, loss_fn, optimizer):
    # Builds function that performs a step in the train loop
    def train_step(data):
        # Sets model to TRAIN mode
        model.train()
        # Makes predictions
        logit, out = model(data.x,data.edge_index)
        # Computes loss
        loss = loss_fn(logit[data.train_mask], data.y[data.train_mask])
        # Computes gradients
        loss.backward()
        # Updates parameters and zeroes gradients
        optimizer.step()
        optimizer.zero_grad()
        # Returns the loss
        return loss.item(), out.tolist()
    
    # Returns the function that will be called inside the train loop
    return train_step

# Creates the train_step function for our model, loss function and optimizer
#train_step = make_train_step(model, loss_fn, optimizer)
#losses = []

# For each epoch...
#for epoch in range(n_epochs):
    # Performs one train step and returns the corresponding loss
    #loss = train_step(x_train_tensor, y_train_tensor)
    #losses.append(loss)
    

In [13]:
weigths = [2.5, 0.625]
imbalance = torch.FloatTensor(weigths).to(device)

In [13]:
#NOT NEEDED
def argmax_list(list2argmax):
  list2return = []

  for i in list2argmax:
    a = i[0]
    b = i[1]
    if a<=b:
      list2return.append(1)
    else:
      list2return.append(0)

  return list2return

In [14]:
import numpy as np

def metrics_computation(truth,pred_probs):
  print('METRICS ',len(pred_probs),len(truth),type(truth),type(pred_probs))
  #print(pred_probs[0],truth[0])
  #print(type(argmax_res),argmax_res.shape)
  #FOR AUCROC
  probs_tensor = torch.from_numpy(pred_probs)
  #print('tensor probs ',probs_tensor.shape)
  #FOR THE REST OF METRICS
  argmax_res = torch.argmax(probs_tensor,dim=1)
  #print('argmax ',argmax_res.shape)
  #FOR AUC AND REST TRYTH TENSOR
  truth_tensor = np.array(truth)
  truth_tensor = torch.from_numpy(truth_tensor)
  #print('TRUTH ',truth_tensor.shape)

  #COMPUTE AUC ROC SCORE
  auc_score = auc_roc(probs_tensor,truth_tensor)
  #print('AUC SCORE ', auc_score)

  #COMPUTE ACCURACY
  acc = accuracy_score(argmax_res,truth_tensor)
  #print('ACCURACY SCORE',acc)

  #COMPUTE RECALL FOR BOTH CLASSES
  rec = brecall(argmax_res,truth_tensor)
  #print('RECALL SCORE ',rec) 

  #COMPUTE PRECISSION FOR BOTH CLASSES 
  pre = bprecision(argmax_res,truth_tensor)
  #print('PRECISSION SCORE ',pre)

  #COMPUTE F1 SCORE FOR BOTH CLASSES
  f1_score = f1score(argmax_res,truth_tensor)
  #print('F1 SCORE ',f1_score)

  #COMPUTE CONFUSION MATRIX
  conf_m = conf_matrix(argmax_res,truth_tensor)
  #print('CONFUSION MATRIX ', conf_m)

  return auc_score, acc, rec, pre, f1_score, conf_m



In [20]:
#TRAIN WITH DATA OBJECT
def train2(model,epochs,weigths):

  if weigths == True:

    print('WEIGTHS = ',weigths)
    criterion = torch.nn.CrossEntropyLoss(weight=imbalance)
  else:
    criterion = torch.nn.CrossEntropyLoss()

  train_step = make_train_step(model, criterion, model.optimizer) 

  losses = []
  val_losses = []
  best_model = copy.deepcopy(model)
  best_val_loss = 10000

  #METRICS VAL
  accuracy_v = []
  auc_v = []
  recall_score_v = []
  precission_score_v = []
  f1_v = []
  
  print('WORKING WITH TRAIN2')
  for epoch in range(epochs):
    
        
        #NA TO GYRISW SE SOFTMAX ANTI LOG
        loss, probs_train = train_step(data)
        losses.append(loss)

        #print('TRAIN TENSOR ',type(probs_train),len(probs_train),probs_train[0])
        #print('PROBS TRAIN',probs_train[0])
        #tens = torch.tensor([0,1,1,1,0,0])
        #pred_train  = argmax_list(probs_train)

        train_truth = data2.y[train_index].tolist()
        #print('train truth', type(train_truth))

        #print('SIZE ',len(pred),type(pred))
        #pred = torch.tensor(pred)
        #print('SIZE ',len(pred),type(pred),pred.dtype)
        #pred = pred[train_index]
        #print('SIZE ',len(pred),type(pred),pred,pred.dtype)
        #print('B ',len(b),type(b),b.dtype)
        #print('LOSS',loss)
        
    #print('TEST ',type(probs),len(probs),probs[0])
    
        with torch.no_grad():
            
            model.eval()

            logit, out = model(data.x,data.edge_index)
            val_loss = criterion(logit[data.val_mask], data.y[data.val_mask])
            val_losses.append(val_loss.item())

            #print('PROBS VAL ',out.tolist()[0])

            val_probs = out.tolist()
            val_probs = np.array(val_probs)
            val_probs = val_probs[val_index]

            probs_train = np.array(probs_train)
            probs_train = probs_train[train_index]
            

            val_truth = data2.y[val_index].tolist()
            auc_score_val, acc_val, rec_val, pre_val, f1_score_val, conf_m_val = metrics_computation(val_truth,val_probs)
            auc_score_train, acc_train, rec_train, pre_train, f1_score_train, conf_m_train = metrics_computation(train_truth,probs_train)
            
            print('AUC VAL',auc_score_val,' ACC VAL ',acc_val,' RECALL VAL ',rec_val, 'PRECI VAL ', pre_val,' F1 VAL ', f1_score_val,' CM VAL ',conf_m_val)
            print('AUC TRAIN',auc_score_train,' ACC TRAIN ',acc_train,' RECALL TRAIN ',rec_train, 'PRECI TRAIN ', pre_train,' F1 TRAIN ', f1_score_train,' CM TRAIN ',conf_m_train)

        accuracy_v.append(acc_val.item())
        auc_v.append(auc_score_val.item())
        recall_score_v.append(rec_val.tolist())
        precission_score_v.append(pre_val.tolist())
        f1_v.append(f1_score_val.tolist())

        if val_loss < best_val_loss:
          print('LOSSES',val_loss.item(),best_val_loss)
          best_model = copy.deepcopy(model)
          best_val_loss = copy.deepcopy(val_loss.item())
          print('NEW BEST///')     
  return losses, val_losses, best_model, accuracy_v, auc_v, recall_score_v, precission_score_v, f1_v 

In [22]:
#TRAIN WITH LOADER
def train1(model,epochs,weigths):

  if weigths == True:

    print('WEIGTHS = ',weigths)
    criterion = torch.nn.CrossEntropyLoss(weight=imbalance)
  else:
    criterion = torch.nn.CrossEntropyLoss()

  train_step = make_train_step(model, criterion, model.optimizer) 

  losses = []
  val_losses = []
  best_model = copy.deepcopy(model)
  best_val_loss = 10000

  #METRICS VAL
  accuracy_v = []
  auc_v = []
  recall_score_v = []
  precission_score_v = []
  f1_v = []
  

  for epoch in range(epochs):
    for batch in train_loader:
        batch = batch.to(device)
        #NA TO GYRISW SE SOFTMAX ANTI LOG
        loss, probs_train = train_step(batch)
        losses.append(loss)

        #print('TRAIN TENSOR ',type(probs_train),len(probs_train),probs_train[0])
        #print('PROBS TRAIN',probs_train[0])
        #tens = torch.tensor([0,1,1,1,0,0])
        #pred_train  = argmax_list(probs_train)

        train_truth = data2.y[train_index].tolist()
        #print('train truth', type(train_truth))

        #print('SIZE ',len(pred),type(pred))
        #pred = torch.tensor(pred)
        #print('SIZE ',len(pred),type(pred),pred.dtype)
        #pred = pred[train_index]
        #print('SIZE ',len(pred),type(pred),pred,pred.dtype)
        #print('B ',len(b),type(b),b.dtype)
        #print('LOSS',loss)
        
    #print('TEST ',type(probs),len(probs),probs[0])
    
    with torch.no_grad():
        for batch in train_loader:
            batch = batch.to(device)
            
            model.eval()

            logit, out = model(batch.x,batch.edge_index)
            val_loss = criterion(logit[batch.val_mask], batch.y[batch.val_mask])
            val_losses.append(val_loss.item())

            #print('PROBS VAL ',out.tolist()[0])

            val_probs = out.tolist()
            val_probs = np.array(val_probs)
            val_probs = val_probs[val_index]

            probs_train = np.array(probs_train)
            probs_train = probs_train[train_index]
            

            val_truth = data2.y[val_index].tolist()
            auc_score_val, acc_val, rec_val, pre_val, f1_score_val, conf_m_val = metrics_computation(val_truth,val_probs)
            auc_score_train, acc_train, rec_train, pre_train, f1_score_train, conf_m_train = metrics_computation(train_truth,probs_train)
            
            print('AUC VAL',auc_score_val,' ACC VAL ',acc_val,' RECALL VAL ',rec_val, 'PRECI VAL ', pre_val,' F1 VAL ', f1_score_val)
            print('AUC TRAIN',auc_score_train,' ACC TRAIN ',acc_train,' RECALL TRAIN ',rec_train, 'PRECI TRAIN ', pre_train,' F1 TRAIN ', f1_score_train)

        accuracy_v.append(acc_val.item())
        auc_v.append(auc_score_val.item())
        recall_score_v.append(rec_val.tolist())
        precission_score_v.append(pre_val.tolist())
        f1_v.append(f1_score_val.tolist())

        if val_loss < best_val_loss:
          print('LOSSES',val_loss.item(),best_val_loss)
          best_model = copy.deepcopy(model)
          best_val_loss = copy.deepcopy(val_loss.item())
          print('NEW BEST///')     
  return losses, val_losses, best_model, accuracy_v, auc_v, recall_score_v, precission_score_v, f1_v 

In [27]:
#TEST FUNCTION
def test(model, data):
    """Evaluate the model on test set and print the accuracy score."""
    with torch.no_grad():
      model.eval()
      _, out = model(data.x, data.edge_index)
      test_probs = out.tolist()
      test_probs = np.array(test_probs)
      test_probs = test_probs[test_index]

      test_truth = data2.y[test_index].tolist()

      auc_score_test, acc_test, rec_test, pre_test, f1_score_test, conf_m_test = metrics_computation(test_truth,test_probs)



    return auc_score_test.item(), acc_test.item(), rec_test.tolist(), pre_test.tolist(), f1_score_test.tolist(), conf_m_test.tolist()

In [ ]:
gat6 = GAT2(165,64,2,8).to(device)
los, val_los, bm, a, au, r, p, f = train2(gat6,5,True)

In [28]:
a,b,c,d,e,f = test(bm,data)

METRICS  29064 29064 <class 'list'> <class 'numpy.ndarray'>


In [29]:
print(f)

[[24, 2521], [33, 26486]]


In [ ]:
gat = GAT2(165,64,2,8).to(device)
loss_fn = torch.nn.CrossEntropyLoss()

train_step = make_train_step(gat, loss_fn, gat.optimizer)

losses = []
val_losses = []
n_epochs = 10

for epoch in range(n_epochs):
    for batch in train_loader:
        batch = batch.to(device)
        
        loss, probs = train_step(batch)
        losses.append(loss)
        
    with torch.no_grad():
        for batch in train_loader:
            batch = batch.to(device)
            
            gat.eval()

            logit, out = gat(batch.x,batch.edge_index)
            val_loss = loss_fn(logit[batch.val_mask], batch.y[batch.val_mask])
            val_losses.append(val_loss.item())

In [ ]:
print(losses)
print(val_losses)
print(probs[10])

[1.0175076723098755, 1.4170243740081787, 1.614760398864746, 1.2547857761383057, 0.9136817455291748, 0.6651911735534668, 0.9412901401519775, 0.8531443476676941, 0.6736094951629639, 0.6551904082298279]
[0.6734026074409485, 0.7447313070297241, 0.6016190052032471, 0.39293423295021057, 0.3158823847770691, 0.832743763923645, 0.758849561214447, 0.41440466046333313, 0.2922835350036621, 0.2934720814228058]
[-5.923264503479004, -0.002680045086890459]


In [ ]:
import numpy as np
##########DOULEUEI NA KANW THN LISTA NUMPY ARRAY KAI NA PARW TA INDEXES 
list_one = [3,4,5,6,7,8,9]
array = np.array(list_one)
print(array)
index = [0,3,4]
array2 = array[index]
print(array2)


[3 4 5 6 7 8 9]
[3 6 7]


In [ ]:
def argmax_list(list2argmax):
  list2return = []

  for i in list2argmax:
    a = i[0]
    b = i[1]
    if a<=b:
      list2return.append(1)
    else:
      list2return.append(0)

  return list2return


In [ ]:
list_one = [[0.2,0.1],[0.4,0.6],[0.6,0.8],[0.4,0.6]]
index = [0,1]
res = argmax_list(list_one)
print(res)
res2 = res[0,1]
print(res2)

[0, 1, 1, 1]


TypeError: ignored